In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f transport-all-comments.csv.xz || wget  https://github.com/heiseacademy/ml-python/raw/main/06-ml-projekt/transport-all-comments.csv.xz")

In [ ]:
import pandas as pd

posts = pd.read_csv("transport-all-comments.csv.xz", parse_dates=["created_utc"])

# damit du leichter addieren kannst
posts["count"] = 1

In [ ]:
posts_day = posts.set_index("created_utc")[["count"]].resample("D").sum()
posts_day

In [ ]:
posts_day.plot.line()

In [ ]:
posts_month = posts.set_index("created_utc")[["count"]].resample("M").sum()
posts_month.plot()

In [ ]:
from scipy.stats import linregress
lr = linregress(range(len(posts_month)), posts_month["count"].values)
lr

In [ ]:
import numpy as np
posts_month["count_trend"] = lr.intercept + lr.slope*np.arange(len(posts_month))
posts_month[["count", "count_trend"]].plot.line()

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

In [ ]:
df = pd.DataFrame({"ds": posts_month.index.values, 
                   "y": posts_month["count"].values})
df

In [ ]:
m = Prophet()
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=24, freq='M')
future

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
df = pd.DataFrame({"ds": posts_day.index, "y": posts_day["count"]})

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=731, freq='D')
future

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)